In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Parsing ImpactT.in

Here are some examples of the low level routines to parse the main input file.

Note that the Impact class does this automatically


In [2]:
from impact.parsers import *

In [3]:
from impact.lattice import *

In [4]:
with open('../templates/lcls_injector/ImpactT.in', 'r') as f:    
    data = f.read()
    LINES = data.split('\n')

In [5]:
parse_header(LINES)

{'Npcol': 8,
 'Nprow': 8,
 'Dt': 5e-13,
 'Ntstep': 1000000,
 'Nbunch': 1,
 'Dim': 6,
 'Np': 300000,
 'Flagmap': 1,
 'Flagerr': 0,
 'Flagdiag': 2,
 'Flagimg': 1,
 'Zimage': 0.02,
 'Nx': 64,
 'Ny': 64,
 'Nz': 64,
 'Flagbc': 1,
 'Xrad': 0.015,
 'Yrad': 0.015,
 'Perdlen': 45.0,
 'Flagdist': 16,
 'Rstartflg': 0,
 'Flagsbstp': 0,
 'Nemission': 400,
 'Temission': 6.515803466731775e-12,
 'sigx(m)': 0.0006,
 'sigpx': 0.0,
 'muxpx': 0.0,
 'xscale': 1.0,
 'pxscale': 1.0,
 'xmu1(m)': 0.0,
 'xmu2': 0.0,
 'sigy(m)': 0.0006,
 'sigpy': 0.0,
 'muxpy': 0.0,
 'yscale': 1.0,
 'pyscale': 1.0,
 'ymu1(m)': 0.0,
 'ymu2': 0.0,
 'sigz(m)': 1.27e-06,
 'sigpz': 0.0,
 'muxpz': 0.0,
 'zscale': 1.0,
 'pzscale': 1.0,
 'zmu1(m)': 0.0,
 'zmu2': 0.0,
 'Bcurr': 0.7140000000000003,
 'Bkenergy': 1.0,
 'Bmass': 511005.0,
 'Bcharge': -1.0,
 'Bfreq': 2856000000.0,
 'Tini': -3.249141278122655e-12}

In [6]:
# Find index of the line where the lattice starts
IX_LATTICE = ix_lattice(LINES)

# Gather lattice lines
LATLINES = LINES[IX_LATTICE:]

In [7]:
# Summary of elements in this lattice
def myf(t): return [l for l in LATLINES if parse_type(l) == t ] 
quads = myf('quadrupole')
#drifts = myf('drift')
solrfs = myf('solrf')
wakefields = myf('wakefield')

alltypes = set([parse_type(l) for l in LATLINES])
print(alltypes)

for t in ele_type.values():
    l = len(myf(t))
    if l > 0:
        print(l, t)

{'rotationally_symmetric_to_3d', 'quadrupole', 'write_beam', 'wakefield', 'solrf', 'spacecharge', 'change_timestep', 'stop', 'comment'}
8 quadrupole
10 solrf
4 write_beam
1 change_timestep
1 rotationally_symmetric_to_3d
2 wakefield
1 spacecharge
1 stop


# Individual element parsing

In [8]:
parse_quadrupole(quads[0])

{'zedge': 0.01601,
 'b1_gradient': 0.0,
 'L_effective': 0.21,
 'radius': 0.0254,
 'x_offset': 0.0,
 'y_offset': 0.0,
 'x_rotation': 0.0,
 'y_rotation': 0.0,
 'z_rotation': 0.7853981633974483}

In [9]:
parse_solrf(solrfs[0])

{'zedge': 0.0,
 'rf_field_scale': 47537665.059089914,
 'rf_frequency': 2856000000.0,
 'theta0_deg': 303.93723122804266,
 'filename': 'rfdata201',
 'radius': 0.15,
 'x_offset': 0.0,
 'y_offset': 0.0,
 'x_rotation': 0.0,
 'y_rotation': 0.0,
 'z_rotation': 0.0,
 'solenoid_field_scale': 0.0}

In [10]:
parse_wakefield(wakefields[0])

{'s_begin': 1.485,
 's': 4.527856,
 'method': 'analytical',
 'iris_radius': 0.0116,
 'gap': 0.0292,
 'period': 0.035}

# Full lattice parsing into element dicts

In [11]:
from impact.lattice import ele_str

In [12]:
# This parses all lines. 
eles = parse_lattice(LATLINES)

In [13]:
# MAD-style printing
for e in eles:
    s = ele_str(e)
    print(s)

change_timestep_1: change_timestep, description=name:change_timestep_1, dt=4e-12,
      s=0.25
SC_2D_to_3D: rotationally_symmetric_to_3d, description=name:SC_2D_to_3D, s=-1000.0

!__________________ 1.5 cell gun __________________
! Single particle phased and scaled for 6 MeV energy
GUN: solrf, description=name:GUN, L=0.15, zedge=0.0, rf_field_scale=47537665.059089914,
      rf_frequency=2856000000.0, theta0_deg=303.93723122804266, filename=rfdata201,
      radius=0.15, x_offset=0.0, y_offset=0.0, x_rotation=0.0, y_rotation=0.0,
      z_rotation=0.0, solenoid_field_scale=0.0, s=0.15

!__________________Solenoid and correctors__________________
SOL1: solrf, description=name:SOL1, L=0.49308, zedge=0.0, rf_field_scale=0.0,
      rf_frequency=0.0, theta0_deg=0.0, filename=rfdata102, radius=0.15, x_offset=0.0,
      y_offset=0.0, x_rotation=0.0, y_rotation=0.0, z_rotation=0.0,
      solenoid_field_scale=0.2457, s=0.49308
! Corrector quads: skew and regular. 
! Overlap SOL1. Effective length

In [14]:
# Output lines
for e in eles:
    print(ele_line(e))

0 0 0 -4 0.0 0.0 0.25 4e-12 /!name:change_timestep_1
0 0 0 -5 0.0 0.0 -1000.0 /!name:SC_2D_to_3D

!__________________ 1.5 cell gun __________________
! Single particle phased and scaled for 6 MeV energy
0.15 0 0 105 0.0 47537665.059089914 2856000000.0 303.93723122804266 201 0.15 0.0 0.0 0.0 0.0 0.0 0.0 /!name:GUN

!__________________Solenoid and correctors__________________
0.49308 0 0 105 0.0 0.0 0.0 0.0 102 0.15 0.0 0.0 0.0 0.0 0.0 0.2457 /!name:SOL1
! Corrector quads: skew and regular. 
! Overlap SOL1. Effective length, radius estimated from measurements. 
! Max field should be 0.00714 T/m
0.36 0 0 1 0.01601 0.0 0.21 0.0254 0.0 0.0 0.0 0.0 0.7853981633974483 /!name:SQ01
0.36 0 0 1 0.01601 0.0 0.21 0.0254 0.0 0.0 0.0 0.0 0.0 /!name:CQ01
!________________________________________________________________________

!!! Broken: 0 1 101 -2 0.0 0.0 0.61362 /!name:YAG01
0 1 102 -2 0.0 0.0 1.38841 /!name:YAG02

!__________________ L0A begin __________________
0 -1 0 -6 1 1 1.485 4.527856 0.011